<a href="https://colab.research.google.com/github/yvainetian/CV_learning/blob/master/task5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Task5 模型集成
## 5.1 学习目标

    * 学习集成学习方法以及交叉验证情况下的模型集成
    * 学会使用深度学习模型的集成学习
## 5.2 集成学习方法
    集成学习是一种训练思路--“人多力量大”，它并没有创造出新的算法，而是把已有的算法进行结合，从而得到更好的效果。
    主要组装基础模型的方法：bagging，boosting和Stacking:
### 5.2.1 Bagging
    bagging的核心思路--民主
    每个基础模型都一直对待，通过民主投票的方式得到最终结果。
    从原始样本集中抽取训练集。每轮从原始样本集中使用Bootstraping的方法抽取n个训练样本。共进行k轮抽取，得到k个训练集，k个模型。
    对分类问题：将上步得到的k个模型采用投票的方式得到分类结果；对回归问题，计算上述模型的均值作为最后的结果。（所有模型的重要性相同）一般情况下，经过bagging得到的结果方差更小。
    例子：bagging + 决策树 = 随机森林
### 5.2.2 Boosting
    Boosting 的核心思路是--挑选精英
    差别对待，给精英更多的投票权，表现不好的基础模型则给较少的投票权，然后综合所有人的投票得到最终结果。
    通过加法模型将基础模型进行线性的组合。
    在每一轮改变训练数据的权值或概率分布，通过提高那些在前一轮被弱分类器分错样例的权值，减小前一轮分对样例的权值，来使得分类器对误分的数据有较好的效果。
    经过 boosting 得到的结果偏差更小
    如：AdaBoost 算法
* 二者差别在于：
    bagging：训练集是在原始集中有放回选取，所以相互独立；使用均匀抽样，每个样例权重相等；各预测函数可以并行生成。
    boosting：训练集不变，只是训练集中每个样例在分类器中权重会根据上一轮的分类结果进行调整，也因此预测函数只能顺序生成。

### 5.2.3 Stacking
    可以使用某种「k-折交叉训练」方法，在 k-1 折数据上进行训练，从而对剩下的一折数据进行预测，迭代地重复这个过程；然后使用所有这些预测结果作为输入训练元模型得到最终预测结果。

* 那么在10个CNN模型可以使用如下方式进行集成：
        1. 对预测的结果的概率值进行平均，然后解码为具体字符；
        2. 对预测的字符进行投票，得到最终字符。

    https://www.cnblogs.com/Christina-Notebook/p/10063146.html

## 5.3 深度学习中的集成学习
### 5.3.1 Dropout
    Dropout可以作为训练深度神经网络的一种技巧。在每个训练批次中，通过随机让一部分的节点停止工作。同时在预测的过程中让所有的节点都其作用。
    可以有效的缓解模型过拟合的情况，也可以在预测时增加模型的精度。

In [0]:
# 定义模型
class SVHN_Model1(nn.Module):
    def __init__(self):
        super(SVHN_Model1, self).__init__()
        # CNN提取特征模块
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2)),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2)),
            nn.ReLU(), 
            nn.Dropout(0.25),
            nn.MaxPool2d(2),
        )
        # 
        self.fc1 = nn.Linear(32*3*7, 11)
        self.fc2 = nn.Linear(32*3*7, 11)
        self.fc3 = nn.Linear(32*3*7, 11)
        self.fc4 = nn.Linear(32*3*7, 11)
        self.fc5 = nn.Linear(32*3*7, 11)
        self.fc6 = nn.Linear(32*3*7, 11)
    
    def forward(self, img):        
        feat = self.cnn(img)
        feat = feat.view(feat.shape[0], -1)
        c1 = self.fc1(feat)
        c2 = self.fc2(feat)
        c3 = self.fc3(feat)
        c4 = self.fc4(feat)
        c5 = self.fc5(feat)
        c6 = self.fc6(feat)
        return c1, c2, c3, c4, c5, c6

### 5.3.2 TTA
    测试集数据扩增（Test Time Augmentation，简称TTA）
    数据扩增不仅可以在训练时候用，而且可以同样在预测时候进行数据扩增，对同一个样本预测三次，然后对三次结果进行平均。

In [0]:
def predict(test_loader, model, tta=10):
   model.eval()
   test_pred_tta = None
   # TTA 次数
   for _ in range(tta):
       test_pred = []
   
       with torch.no_grad():
           for i, (input, target) in enumerate(test_loader):
               c0, c1, c2, c3, c4, c5 = model(data[0])
               output = np.concatenate([c0.data.numpy(), c1.data.numpy(),
                  c2.data.numpy(), c3.data.numpy(),
                  c4.data.numpy(), c5.data.numpy()], axis=1)
               test_pred.append(output)
       
       test_pred = np.vstack(test_pred)
       if test_pred_tta is None:
           test_pred_tta = test_pred
       else:
           test_pred_tta += test_pred
   
   return test_pred_tta

### 5.3.3 Snapshot
    使用cyclical learning rate进行训练模型，并保存精度比较好的一些checkopint，最后将多个checkpoint进行模型集成。
    由于在cyclical learning rate中学习率的变化有周期性变大和减少的行为，因此CNN模型很有可能在跳出局部最优进入另一个局部最优。在Snapshot论文中作者通过使用表明，此种方法可以在一定程度上提高模型精度，但需要更长的训练时间。
    https://cloud.tencent.com/developer/article/1160636



## 5.4 结果后处理
    在本次赛题中，可以从以下几个思路对预测结果进行后处理：
    * 统计图片中每个位置字符出现的频率，使用规则修正结果；
    * 单独训练一个字符长度预测模型，用来预测图片中字符个数，并修正结果。
    
* 集成学习只能在一定程度上提高精度，并需要耗费较大的训练时间，因此建议先使用提高单个模型的精度，再考虑集成学习过程

* 具体的集成学习方法需要与验证集划分方法结合，Dropout和TTA在所有场景有可以起作用。

